# Semantic Kernel ツール使用例：ChromaDBによるRAGシステム

## 🎯 このノートブックで学ぶこと

このノートブックでは、**Retrieval-Augmented Generation（RAG）システム**の構築方法を学習します。RAGとは、大量のドキュメントから関連情報を検索し、その情報を活用してより正確な回答を生成するAI技術です。

### 💡 実装する機能
- **ChromaDB**: ベクトルデータベースとして旅行関連文書を保存・検索
- **Semantic Kernel**: マイクロソフトのAIエージェントフレームワーク
- **複数プラグイン**: 文書検索、天気情報、目的地詳細の各種ツール
- **ストリーミング応答**: リアルタイムでAIの思考過程を確認

### 🚀 完成イメージ
「モルディブの気温と旅行保険について教えて」という質問に対して、AIエージェントが：
1. ChromaDBから旅行保険の情報を検索
2. 天気プラグインからモルディブの気温データを取得
3. 両方の情報を統合して詳細な回答を生成

### 📚 技術スタック
- **ChromaDB**: ローカルベクトルデータベース
- **Semantic Kernel**: AIエージェントオーケストレーション
- **OpenAI GPT-4o-mini**: GitHub Models経由で言語モデルを使用

## 環境の初期化

このセクションでは、RAGシステムを動作させるために必要な環境設定を行います。主に以下の準備を行います：

1. **SQLiteバージョン対応**: ChromaDBが要求するSQLiteバージョンの確保
2. **パッケージインストール**: 必要なPythonライブラリの自動インストール
3. **ライブラリインポート**: Semantic Kernel、ChromaDB、OpenAIライブラリの読み込み

### 🔧 SQLiteバージョン修正（トラブルシューティング）

ChromaDBはベクトルデータベースとしてSQLiteを使用しますが、古いバージョンでは動作しません。

**エラーが発生する場合：**
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```

**解決方法：** 下記のセルのコメント（#）を外して実行してください：
- `pysqlite3-binary`をインストール
- システムのsqlite3モジュールを新しいバージョンで置き換え

In [1]:
# %pip install pysqlite3-binary
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### 📦 必要なパッケージのインストール

このセクションでは、RAGシステムに必要なPythonライブラリを自動的にインストールします。

**インストール対象：**
- `chromadb`: ベクトルデータベース（文書の埋め込み保存・検索）
- `semantic-kernel`: Microsoftのマルチエージェントフレームワーク
- `openai`: OpenAI APIクライアント（GitHub Models経由）
- `python-dotenv`: 環境変数管理（APIキー等）
- `ipython`: Jupyter Notebook表示機能

**スマート機能：** 既にインストール済みのパッケージはスキップして実行時間を短縮します。

In [2]:
# 必要なパッケージのインストール（未インストールの場合のみ）
import importlib
import subprocess
import sys

def install_if_missing(package_name, import_name=None):
    """パッケージが見つからない場合のみインストール"""
    if import_name is None:
        import_name = package_name
    
    try:
        importlib.import_module(import_name)
        print(f"✅ {package_name} は既にインストールされています")
    except ImportError:
        print(f"📦 {package_name} をインストール中...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"✅ {package_name} のインストールが完了しました")

# 必要なパッケージをチェック＆インストール
packages = [
    ("chromadb", "chromadb"),
    ("semantic-kernel", "semantic_kernel"),
    ("openai", "openai"),
    ("python-dotenv", "dotenv"),
    ("ipython", "IPython")
]

for package, import_name in packages:
    install_if_missing(package, import_name)

📦 chromadb をインストール中...
✅ chromadb のインストールが完了しました
📦 semantic-kernel をインストール中...
✅ semantic-kernel のインストールが完了しました
✅ openai は既にインストールされています
✅ python-dotenv は既にインストールされています
✅ ipython は既にインストールされています


In [3]:
import json
import os
import chromadb
from typing import Annotated, TYPE_CHECKING

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent,FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

if TYPE_CHECKING:
    from chromadb.api.models.Collection import Collection

### 🤖 Semantic KernelとAIサービスの作成

ここでは、AIエージェントの「脳」となる言語モデルを設定します。

**設定内容：**
- **OpenAIクライアント**: GitHub Models（無料）を通じてGPT-4o-miniにアクセス
- **APIキー**: 環境変数`GITHUB_TOKEN`から取得（GitHub Personal Access Tokenを使用）
- **Semantic Kernel**: Microsoftのマルチエージェントフレームワークを初期化

**重要：** GitHub Personal Access TokenにModels権限が必要です（Settings > Developer settings > Personal access tokens）

In [4]:
# 非同期OpenAIクライアントの初期化
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com/"
)


# OpenAI チャット完了サービスの作成
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

### 🔍 プロンプトプラグインの定義（RAGの中核機能）

**このプラグインの役割：**
RAGシステムの心臓部となる文書検索機能を実装します。ユーザーの質問に関連する情報をChromaDBから自動的に取得します。

**実装機能：**
1. **`retrieve_context`**: ChromaDBからセマンティック検索を実行
   - ユーザーの質問をベクトル化
   - 類似度の高い文書を上位2件取得
   - メタデータと合わせて構造化して返却

2. **`build_augmented_prompt`**: 検索結果を使って拡張プロンプトを作成
   - 取得した文書コンテキストとユーザー質問を組み合わせ
   - AIが正確な情報に基づいて回答できるようガイド

**RAGの流れ：** 質問 → ベクトル検索 → 関連文書取得 → コンテキスト付き回答生成

In [5]:
class PromptPlugin:

    def __init__(self, collection: "Collection"):
        self.collection = collection

    @kernel_function(
        name="build_augmented_prompt",
        description="取得コンテキストを使用して拡張プロンプトを構築します。"
    )
    def build_augmented_prompt(self, query: str, retrieval_context: str) -> str:
        return (
            f"取得されたコンテキスト:\n{retrieval_context}\n\n"
            f"ユーザークエリ: {query}\n\n"
            "上記のコンテキストのみに基づいて、回答を提供してください。"
        )
    
    @kernel_function(name="retrieve_context", description="データベースからコンテキストを取得します。")
    def get_retrieval_context(self, query: str) -> str:
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2
        )
        context_entries = []
        if results and results.get("documents") and results["documents"][0]:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_entries.append(f"文書: {doc}\nメタデータ: {meta}")
        return "\n\n".join(context_entries) if context_entries else "取得可能なコンテキストが見つかりませんでした。"

### 🌤️ 天気情報プラグインの定義（外部データ統合）

**このプラグインの役割：**
旅行計画に重要な気象情報を提供する専門ツールです。RAGシステムに外部データソースとの連携機能を追加します。

**機能詳細：**
- **対象地域**: モルディブ、スイスアルプス、アフリカンサファリ
- **提供情報**: 平均気温（摂氏・華氏両方）
- **エラーハンドリング**: 未対応地域の場合は利用可能地域を案内

**実際の運用では：**
- 気象API（OpenWeatherMap等）と連携
- リアルタイム天気データの取得
- 予報情報や警報の統合

**RAGとの連携：** 文書検索 + リアルタイム外部データ = より包括的な回答

In [6]:
class WeatherInfoPlugin:
    """旅行先の平均気温を提供するプラグイン。"""

    def __init__(self):
        # 目的地とその平均気温の辞書
        self.destination_temperatures = {
            "maldives": "28°C (82°F)",
            "swiss alps": "7°C (45°F)", 
            "african safaris": "24°C (75°F)"
        }

    @kernel_function(description="特定の旅行先の平均気温を取得します。")
    def get_destination_temperature(self, destination: str) -> Annotated[str, "目的地の平均気温を返します。"]:
        """旅行先の平均気温を取得します。"""
        # 入力された目的地を正規化（小文字）
        normalized_destination = destination.lower()

        # 目的地の気温を検索
        if normalized_destination in self.destination_temperatures:
            return f"{destination}の平均気温は{self.destination_temperatures[normalized_destination]}です。"
        else:
            return f"申し訳ございません、{destination}の気温情報がありません。利用可能な目的地は：モルディブ、スイスアルプス、アフリカンサファリです。"

### 🗺️ 目的地情報プラグインの定義（構造化知識ベース）

**このプラグインの役割：**
旅行先の詳細な構造化情報を提供する知識ベースです。ChromaDBの文書検索を補完する専門データソースとして機能します。

**包含する詳細情報：**
- **基本情報**: 地域名、概要説明
- **旅行計画**: ベストシーズン、平均コスト
- **アクティビティ**: 人気の観光・体験メニュー
- **対象地域**: モルディブ、スイスアルプス、アフリカンサファリ、バリ、サントリーニ

**高度な検索機能：**
- **キーワードマッチング**: 英語名・日本語名の両方に対応
- **部分一致検索**: 「サファリ」→「アフリカンサファリ」に自動拡張
- **フォールバック**: 該当なしの場合は一般検索システムに移行

**RAGアーキテクチャでの位置づけ：**
- ChromaDB: 一般的な旅行文書（保険、サービス等）
- 目的地プラグイン: 特定地域の専門的構造化データ
- 天気プラグイン: リアルタイム気象情報

In [7]:
class DestinationsPlugin:
    # 人気の旅行地に関する豊富な詳細情報を持つ目的地データストア
    DESTINATIONS = {
        "maldives": {
            "name": "モルディブ",
            "description": "インド洋に浮かぶ26の環礁からなる群島で、pristine beachesと水上バンガローで知られています。",
            "best_time": "11月から4月（乾季）",
            "activities": ["シュノーケリング", "ダイビング", "アイランドホッピング", "スパリトリート", "水中ダイニング"],
            "avg_cost": "高級リゾートで1泊400-1200ドル"
        },
        "swiss alps": {
            "name": "スイスアルプス",
            "description": "スイス全土にまたがる山脈で、絵のような村と世界クラスのスキーリゾートがあります。",
            "best_time": "スキーは12月から3月、ハイキングは6月から9月",
            "activities": ["スキー", "スノーボード", "ハイキング", "マウンテンバイク", "パラグライディング"],
            "avg_cost": "アルペン宿泊施設で1泊250-500ドル"
        },
        "safari": {
            "name": "アフリカンサファリ",
            "description": "ケニア、タンザニア、南アフリカなど様々なアフリカ諸国での野生動物観察体験。",
            "best_time": "最適な野生動物観察のため6月から10月（乾季）",
            "activities": ["ゲームドライブ", "ウォーキングサファリ", "熱気球ライド", "文化村訪問"],
            "avg_cost": "高級サファリパッケージで1人1日400-800ドル"
        },
        "bali": {
            "name": "バリ、インドネシア",
            "description": "豊かな棚田、美しい寺院、活気ある文化で知られる島の楽園。",
            "best_time": "4月から10月（乾季）",
            "activities": ["サーフィン", "寺院見学", "棚田トレッキング", "ヨガリトリート", "ビーチでのリラクゼーション"],
            "avg_cost": "宿泊タイプに応じて1泊100-500ドル"
        },
        "santorini": {
            "name": "サントリーニ、ギリシャ",
            "description": "エーゲ海を見下ろす白い建物と青いドームで美しい火山島。",
            "best_time": "4月下旬から11月上旬",
            "activities": ["イアでの夕日鑑賞", "ワインテイスティング", "ボートツアー", "ビーチホッピング", "古代遺跡探索"],
            "avg_cost": "カルデラビューの宿泊施設で1泊200-600ドル"
        }
    }

    @kernel_function(
        name="get_destination_info",
        description="特定の旅行先に関する詳細情報を提供します。"
    )
    def get_destination_info(self, query: str) -> str:
        # どの目的地について尋ねられているかを見つける
        query_lower = query.lower()
        matching_destinations = []

        for key, details in DestinationsPlugin.DESTINATIONS.items():
            if key in query_lower or details["name"].lower() in query_lower:
                matching_destinations.append(details)

        if not matching_destinations:
            return (f"ユーザークエリ: {query}\n\n"
                    f"データベースで特定の目的地情報が見つかりませんでした。"
                    f"このクエリには一般的な取得システムを使用してください。")

        # 目的地情報のフォーマット
        destination_info = "\n\n".join([
            f"目的地: {dest['name']}\n"
            f"説明: {dest['description']}\n"
            f"ベストシーズン: {dest['best_time']}\n"
            f"人気のアクティビティ: {', '.join(dest['activities'])}\n"
            f"平均コスト: {dest['avg_cost']}" for dest in matching_destinations
        ])

        return (f"目的地情報:\n{destination_info}\n\n"
                f"ユーザークエリ: {query}\n\n"
                "上記の目的地詳細に基づいて、この場所に関するユーザーのクエリに"
                "対応する有用な回答を提供してください。")

## 💾 ChromaDBのセットアップ（ベクトルデータベース構築）

**このセクションの目的：**
RAGシステムの情報源となるベクトルデータベースを構築します。テキスト文書を数値ベクトル（埋め込み）に変換して保存し、セマンティック検索を可能にします。

**実装内容：**

1. **永続ストレージ**: `./chroma_db`フォルダにデータを保存（再起動後もデータ保持）
2. **コレクション作成**: `travel_documents`という名前の文書コレクションを作成
3. **サンプルデータ投入**: Contoso Travel社の旅行サービス情報を5件追加

**投入する文書内容：**
- 高級バケーションパッケージの概要
- プレミアム旅行サービスの詳細
- 旅行保険の適用範囲
- 人気観光地の紹介
- 特別サービス（ブティックホテル、ガイドツアー）

**メタデータ管理：**
各文書に`source: training`、`type: explanation`のメタデータを付与し、検索時の文脈理解を向上させます。

**ベクトル検索の仕組み：**
ユーザーの質問 → 質問をベクトル化 → 類似ベクトルを持つ文書を検索 → 関連文書を返却

In [8]:
collection = chromadb.PersistentClient(path="./chroma_db").create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True,
)

documents = [
    "Contoso Travelは世界中のエキゾチックな目的地への高級バケーションパッケージを提供しています。",
    "当社のプレミアム旅行サービスには、個別の旅程計画と24時間365日のコンシェルジュサポートが含まれています。",
    "Contosoの旅行保険は、医療緊急事態、旅行キャンセル、手荷物紛失をカバーしています。",
    "人気の目的地には、モルディブ、スイスアルプス、アフリカンサファリなどがあります。",
    "Contoso Travelは、ブティックホテルや専属ガイド付きツアーへの特別アクセスを提供しています。",
]

collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

In [9]:
agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[DestinationsPlugin(), WeatherInfoPlugin(), PromptPlugin(collection)],
    name="TravelAgent",
    instructions="提供されたコンテキストを使用して旅行に関するクエリに回答してください。コンテキストが提供されている場合、「そのためのコンテキストがありません」とは言わないでください。",
)

### 🤖 AIエージェントの作成（全機能統合）

**このセクションの目的：**
これまでに定義した全ての機能を統合し、実際に動作するRAGエージェントを作成します。

**エージェント構成：**
- **AIサービス**: GitHub Models経由のGPT-4o-mini
- **プラグイン統合**: 
  - 目的地情報プラグイン（構造化データ）
  - 天気情報プラグイン（外部データ）
  - プロンプトプラグイン（ChromaDB検索）
- **名前**: "TravelAgent"（旅行専門アシスタント）
- **指示**: コンテキスト重視の回答生成

**動作フロー：**
1. ユーザー質問を受信
2. 各プラグインが並列で関連情報を収集
3. 収集した情報を統合して回答を生成
4. ソース情報とともに回答を提示

### 🚀 RAGエージェントのデモ実行（リアルタイム対話体験）

**このセクションの目的：**
作成したRAGエージェントを実際に動作させ、3つのテストケースでその能力を検証します。

**テストケースと期待される動作：**

1. **📄 文書検索テスト**: 「Contosoの旅行保険について」
   ```
   関数呼び出し: retrieve_context({"query": "Contoso旅行保険"})
   関数結果: 文書: Contosoの旅行保険は、医療緊急事態、旅行キャンセル、手荷物紛失をカバーしています。
   
   TravelAgent: Contosoの旅行保険は以下をカバーします：
   - 医療緊急事態での治療費
   - 旅行キャンセル時の返金
   - 手荷物紛失の補償...
   ```

2. **🌡️ 外部データテスト**: 「モルディブの気温は？」
   ```
   関数呼び出し: get_destination_temperature({"destination": "maldives"})
   関数結果: モルディブの平均気温は28°C (82°F)です。
   
   TravelAgent: モルディブの平均気温は28°C（82°F）です。一年を通して温暖で...
   ```

3. **🔄 複合クエリテスト**: 「寒い目的地と気温」
   ```
   関数呼び出し: get_destination_info({"query": "寒い目的地"})
   関数呼び出し: get_destination_temperature({"destination": "swiss alps"})
   
   TravelAgent: 寒い目的地としてスイスアルプスをお勧めします。
   平均気温は7°C（45°F）で、スキーやスノーボードに最適です...
   ```

**ストリーミング表示の特徴：**
- **📋 関数呼び出しログ**: クリックで展開可能（AIの思考過程を透明化）
- **🎯 リアルタイム生成**: 回答が段階的に表示される
- **📚 ソース明示**: どの情報源から回答を得たかが明確

---

### 📋 学習のまとめ

**✅ 実装したRAGシステムの特徴：**

1. **📚 多層情報アーキテクチャ**: ChromaDB文書検索 + 構造化プラグイン + 外部データ統合
2. **🤖 インテリジェントな情報統合**: 複数ソースからの並列情報取得とコンテキスト重視の回答生成
3. **🔍 透明な処理過程**: 関数呼び出しログとストリーミング表示による思考過程の可視化

**🚀 実用化展開アイデア：**
- **企業サポート**: 製品マニュアル + FAQ + リアルタイム在庫情報
- **医療システム**: 症状データベース + 最新研究 + 薬剤相互作用チェック
- **教育プラットフォーム**: 教科書内容 + 演習問題 + 学習進捗データ

In [10]:
async def main():
    """RAGエージェントのデモを実行する関数"""
    thread: ChatHistoryAgentThread | None = None

    # テスト用の質問リスト（実際のRAG機能を検証）
    user_inputs = [
        "Contosoの旅行保険の適用範囲について説明してもらえますか？",  # ChromaDB検索テスト
        "モルディブの平均気温はどれくらいですか？",                    # 天気プラグインテスト
        "Contosoが提供する寒い目的地で良い場所と、その平均気温を教えてください。",  # 複合クエリテスト
    ]

    print("🚀 RAGエージェントデモを開始します...\n")
    print("=" * 60)

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>ユーザー:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # ストリーミング関数呼び出しを再構築するバッファ
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # 引数を蓄積（チャンクでストリーミング）
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # 結果を表示する前に保留中の関数呼び出しを終了
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # 生の文字列のまま残す

                        function_calls.append(f"関数呼び出し: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\n関数結果:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        # HTML形式で美しく表示
        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>関数呼び出し（クリックして展開）</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'アシスタント'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

    print("\n✅ デモが完了しました！")

# 🎬 RAGエージェントのデモを実行
print("📋 実行準備:")
print("- GITHUB_TOKEN環境変数の確認")
print("- ChromaDBの初期化")
print("- 全プラグインの準備完了")
print("\n🔄 実行中...")

await main()

📋 実行準備:
- GITHUB_TOKEN環境変数の確認
- ChromaDBの初期化
- 全プラグインの準備完了

🔄 実行中...
🚀 RAGエージェントデモを開始します...




✅ デモが完了しました！
